# COVID-19 Case Counts from Corona Data Scraper

**[Work in progress]**

This notebook creates a .csv file with cummulative confimed cases and deaths for ingestion into the Knowledge Graph.

Data source: [Corona Data Scraper](https://coronadatascraper.com/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import dateutil

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-19636412-9e74-4bac-8a4c-c6c8b49bb9d3/installation-4.1.0/import


In [4]:
def split_by_day(df, day, label):
    day_df = df[['stateFips', 'countyFips', day]].copy()
    day_df.rename(columns={day: label}, inplace=True)
    day_df['date'] = day
    return day_df

### Process cummulative confirmed cases

In [5]:
df = pd.read_csv("https://coronadatascraper.com/timeseries-tidy.csv.zip",  dtype='str')

In [6]:
df = df.fillna('')

In [7]:
df['type'].unique()

array(['cases', 'growthFactor', 'hospitalized_current', 'discharged',
       'deaths', 'tested', 'recovered', 'active', 'hospitalized',
       'icu_current', 'icu'], dtype=object)

In [8]:
def split_by_type(df, value_type, label):
    df_type = df[df['type'] == value_type].copy()
    df_type = df_type[['level', 'country', 'state', 'county', 'date', 'value']]
    df_type.rename(columns={'value': label}, inplace=True)
    return df_type

In [9]:
confirmed = split_by_type(df, 'cases', 'confirmedCases')
confirmed.head()

,level,country,state,county,date,confirmedCases
0,county,Belgium,Flanders,Antwerp,2020-01-22,4
1,county,Belgium,Flanders,Antwerp,2020-01-23,4
3,county,Belgium,Flanders,Antwerp,2020-01-24,4
5,county,Belgium,Flanders,Antwerp,2020-01-25,4
7,county,Belgium,Flanders,Antwerp,2020-01-26,4


In [10]:
deaths = split_by_type(df, 'deaths', 'confirmedDeaths')
deaths.head()

,level,country,state,county,date,confirmedDeaths
2422,state,Belgium,Flanders,,2020-03-11,1
2425,state,Belgium,Flanders,,2020-03-12,1
2428,state,Belgium,Flanders,,2020-03-13,1
2431,state,Belgium,Flanders,,2020-03-14,2
2434,state,Belgium,Flanders,,2020-03-15,5


### Process cummulative deaths

In [11]:
deaths = df.query("type == 'deaths'").copy()
deaths = deaths[['level', 'country', 'state', 'county', 'date', 'value']]
deaths.rename(columns={'value': 'cummulativeDeaths'}, inplace=True)
deaths.head()

,level,country,state,county,date,cummulativeDeaths
2422,state,Belgium,Flanders,,2020-03-11,1
2425,state,Belgium,Flanders,,2020-03-12,1
2428,state,Belgium,Flanders,,2020-03-13,1
2431,state,Belgium,Flanders,,2020-03-14,2
2434,state,Belgium,Flanders,,2020-03-15,5


#### Reformat dataframe by day